<a href="https://colab.research.google.com/github/rafaelpivetta/tech-challenge-fase3/blob/main/fine-tuning/Teste_modelo_tunado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [32]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
model_name = "unsloth/tinyllama-chat-bnb-4bit" # Utilizar este para teste de inferência antes do tuning
# rafaelpivetta/tinyllama-chat-bnb-4bit-g19 #Utilizar este para teste de inferência após o tuning

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.9.post1: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [34]:
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Create a description for the following product: Design-For-Test For Digital IC's and Embedded Core Systems", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs,
                         max_new_tokens = 128,
                         use_cache = True,
                         temperature=0.7,
                         top_p=0.9,
                         repetition_penalty=1.1
                         )
tokenizer.batch_decode(outputs)

["<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a description for the following product: Design-For-Test For Digital IC's and Embedded Core Systems\n\n### Response:\nI am not capable of creating a design for digital ic's and embedded core systems. However, I can provide you with an overview of the product. The design-for-test (dfx) tool is a software solution designed to help developers create test plans and automate testing processes. It provides a comprehensive set of tools for designing, executing, and analyzing tests. The dfx tool includes a range of features such as test case generation, test plan creation, test execution, and reporting. The tool also supports various test frameworks such as unified modeling language (uml), system verification language (sv"]